Making the csvs for pharos

In [1]:
#%pip install --upgrade pip --user
# %pip install pandas==2.2.3
# %pip install SQLAlchemy==2.0.0

# %pip install sqlalchemy --user
# %pip install numpy --user
# %pip install torch 
# %pip install scikit-learn
# %pip install mysql-connector-python --user
# %pip install matplotlib 
# %pip install subword-nmt



In [13]:
import pandas as pd
from sqlalchemy import create_engine

# Define your database connection parameters
host="localhost"
port=3306
user="root"
password="Afu27959"
database="thesisCurrent"
database2="thesisv6.9"

# Create a SQLAlchemy engine
engine1 = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

engine2 = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database2}')

# Query to get data
query1 = """
SELECT uniprot, tdl FROM protein JOIN target ON protein.ID = target.ID;
"""
df1 = pd.read_sql(query1, engine1) #Current
#print(df1)

query2 = """
SELECT uniprot, tdl FROM protein JOIN target ON protein.ID = target.ID;
"""
df2 = pd.read_sql(query2, engine2) #Older

def compare_versions(df1, df2):
    # Merge dataframes on 'uniprot'
    merged = df1.rename(columns={'tdl': 'TDL_current'}).merge(
        df2.rename(columns={'tdl': 'TDL_v6_9'}),
        on='uniprot',
        how='inner'
    )
    
    # Find proteins that were Tdark in v6.9 and reclassified in v6.11
    reclassified = merged[(merged['TDL_v6_9'] == 'Tdark') & (merged['TDL_current'] != 'Tdark')]
    
    return reclassified

# Compare versions
reclassified_proteins = compare_versions(df1, df2)

# Display results
print(f"Number of reclassified proteins: {len(reclassified_proteins)}")
uniprot_list = reclassified_proteins['uniprot'].tolist()

# Format the list for the SQL query
formatted_uniprot_list = "', '".join(uniprot_list)
print(len(uniprot_list))

# Query to get family information from idg_evol table
query_fam = f"""
SELECT DISTINCT uniprot, fam
FROM idg_evol
WHERE uniprot IN ('{formatted_uniprot_list}');
"""

# Fetch the family data from the database
fam_df = pd.read_sql(query_fam, engine1)

# Merge the family data with the reclassified proteins dataframe
reclassified_proteins_with_fam = reclassified_proteins.merge(fam_df, on='uniprot', how='left')

# Display results with the added family column
print(reclassified_proteins_with_fam)
print(f"Number of reclassified proteins: {len(reclassified_proteins_with_fam)}")


Number of reclassified proteins: 1742
1742
     uniprot TDL_current TDL_v6_9  fam
0     A0AV96        Tbio    Tdark  NaN
1     A0AVF1        Tbio    Tdark  NaN
2     A0FGR9        Tbio    Tdark  NaN
3     A0JNW5        Tbio    Tdark  NaN
4     A0PJK1        Tbio    Tdark  NaN
...      ...         ...      ...  ...
1737  Q9Y6I8        Tbio    Tdark  NaN
1738  Q9Y6I9        Tbio    Tdark  NaN
1739  Q9Y6L7       Tchem    Tdark  NaN
1740  Q9Y6Q3        Tbio    Tdark  NaN
1741  Q9Y6V7        Tbio    Tdark  NaN

[1742 rows x 4 columns]
Number of reclassified proteins: 1742


In [14]:
if uniprot_list:
    # Format the list for SQL query
    formatted_uniprot_list = "', '".join(uniprot_list)
    
    # SQL query to get relevant data from the database
    query_data = f"""
    SELECT 
        p.uniprot AS uniprot,
        p.seq AS sequence,
        da.smiles AS smiles,
        da.act_value AS affinity
    FROM 
        protein p
    JOIN 
        drug_activity da ON p.id = da.target_id
    WHERE 
        p.uniprot IN ('{formatted_uniprot_list}')
    UNION ALL

    SELECT 
        p.uniprot AS uniprot,
        p.seq AS sequence,
        ca.smiles AS smiles,
        ca.act_value AS affinity
    FROM 
        protein p
    JOIN 
        target t ON p.id = t.id
    JOIN 
        cmpd_activity ca ON t.id = ca.target_id
    WHERE 
        p.uniprot IN ('{formatted_uniprot_list}')
    """

In [20]:
import os
import pandas as pd
import numpy as np
from stream import drug2emb_encoder, protein2emb_encoder

# Fetch the data from the database
data_df = pd.read_sql(query_data, engine1)
print(len(data_df))

# Rename columns as needed for MolTrans format
data_df = data_df.rename(columns={'smiles': 'SMILES', 'sequence': 'Target Sequence'})

# Select relevant columns
data_df = data_df[['SMILES', 'Target Sequence', 'affinity']]

# Remove rows with NaN or non-string SMILES values
data_df = data_df[data_df['SMILES'].apply(lambda x: isinstance(x, str) and x.strip() != '')]
print(len(data_df))

# Treat all affinities as positive interactions
data_df['Label'] = 1  
#OR THRESHOLD THE AFFINITIES (NEGATIVE SAMPLES REQUIRED)
# affinity_threshold = 7.0 
# data_df['Label'] = (data_df['affinity'] >= affinity_threshold).astype(int)

# Get unique drugs and targets
all_drugs = data_df['SMILES'].unique()
all_targets = data_df['Target Sequence'].unique()

# Create a set of existing positive pairs for quick lookup
positive_pairs = set(zip(data_df['SMILES'], data_df['Target Sequence']))

#Generate negative samples
num_negatives = len(data_df)  # 1:1 ratio of negatives to positives
negative_samples = []
np.random.seed(42)  # For reproducibility

while len(negative_samples) < num_negatives:
    drug = np.random.choice(all_drugs)
    target = np.random.choice(all_targets)
    
    if (drug, target) not in positive_pairs:  # Ensure it's not a known interaction
        negative_samples.append((drug, target, 0))  # Label 0 for negatives
        positive_pairs.add((drug, target))  # Avoid duplicate negatives

# Convert negative samples to a DataFrame
neg_df = pd.DataFrame(negative_samples, columns=['SMILES', 'Target Sequence', 'Label'])

# Concatenate positives and negatives
final_df = pd.concat([data_df[['SMILES', 'Target Sequence', 'Label']], neg_df], ignore_index=True)

# Shuffle the dataset
final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)
#final_df = data_df
# Initialize encoding lists
drug_encodings, protein_encodings = [], []

# Encode drugs and proteins
for _, row in final_df.iterrows():
    drug_encoding, _ = drug2emb_encoder(row['SMILES'])
    protein_encoding, _ = protein2emb_encoder(row['Target Sequence'])
    
    drug_encodings.append(drug_encoding)
    protein_encodings.append(protein_encoding)

# Add encoded data as new columns
final_df['drug_encoding'] = drug_encodings
final_df['protein_encoding'] = protein_encodings

# Define output path and save as CSV
output_dir = 'dataset/pharos4DAVIS'
os.makedirs(output_dir, exist_ok=True)

final_df.to_csv(f'{output_dir}/pharos4DAVIS.csv', index=False)
print(len(final_df))
print("Final Pharos dataset with negative samples saved successfully.")


498
495
990
Final Pharos dataset with negative samples saved successfully.


In [19]:
print(np.unique(final_df['Label'], return_counts=True))

(array([0, 1]), array([495, 495]))


In [21]:
import os
import pandas as pd
import numpy as np
from stream import drug2emb_encoder, protein2emb_encoder

# Fetch the data from the database
data_df = pd.read_sql(query_data, engine1)
print(len(data_df))

# Rename columns as needed for MolTrans format
data_df = data_df.rename(columns={'smiles': 'SMILES', 'sequence': 'Target Sequence'})

# Select relevant columns
data_df = data_df[['SMILES', 'Target Sequence', 'affinity']]

# Remove rows with NaN or non-string SMILES values
data_df = data_df[data_df['SMILES'].apply(lambda x: isinstance(x, str) and x.strip() != '')]
print(len(data_df))

#OR THRESHOLD THE AFFINITIES (NEGATIVE SAMPLES REQUIRED)
affinity_threshold = 7.0 
data_df['Label'] = (data_df['affinity'] >= affinity_threshold).astype(int)

final_df = data_df
# Initialize encoding lists
drug_encodings, protein_encodings = [], []

# Encode drugs and proteins
for _, row in final_df.iterrows():
    drug_encoding, _ = drug2emb_encoder(row['SMILES'])
    protein_encoding, _ = protein2emb_encoder(row['Target Sequence'])
    
    drug_encodings.append(drug_encoding)
    protein_encodings.append(protein_encoding)

# Add encoded data as new columns
final_df['drug_encoding'] = drug_encodings
final_df['protein_encoding'] = protein_encodings

# Define output path and save as CSV
output_dir = 'dataset/pharos4DAVIS'
os.makedirs(output_dir, exist_ok=True)

final_df.to_csv(f'{output_dir}/pharos4DAVIS_allp.csv', index=False)
print(len(final_df))
print("Final Pharos dataset with negative samples saved successfully.")


498
495
495
Final Pharos dataset with negative samples saved successfully.


In [22]:
print(np.unique(final_df['Label'], return_counts=True))

(array([0, 1]), array([ 76, 419]))
